In [84]:
from autophrase import spacy_txt
import pandas as pd
from snapy import MinHash, LSH
import time
import random
import re
import copy

In [18]:
path = "DEA"
s = spacy_txt()
doc, e, n= s.read_file(path)

gg
August 06, 2020
August 06, 2020 DATE
Contact
Phone Number
IMMEDIATE RELEASE
fentanyl-related overdose deaths
officials
public warning
Counterfeit Pills
Fentanyl
Counterfeit Pills
Fentanyl
SAN DIEGO
Law enforcement officials
the county
the public
a sharp increase
overdose deaths
the highly potent and often deadly drug
fentanyl
SAN DIEGO ORG
152 fentanyl-related overdose deaths
San Diego County
the first six months of this year
203 fentanyl-related deaths
pending confirmation
This year
deaths
all areas
the county
Victims
the average age
2019 DATE
San Diego County GPE
the first six months of this year DATE
119 CARDINAL
another 84 CARDINAL
This year DATE
age from 17 DATE
37 DATE
fentanyl-laced drugs
the DEA
San Diego County District Attorney
U.S. Attorney
San Diego County Sheriff’s Department
San Diego Police Department
San Diego County Health and Human Services
San Diego County Medical Examiner
the DEA ORG
San Diego County Sheriff’s Department ORG
San Diego Police Department ORG
San Di

Feb. 7, 2019
agents
officers
NY Port Authority Police Department Task Force Officer
K-9 Balu
surveillance
an ongoing investigation
a suspected narcotics trafficker
Andres Reyes-Martinez
a cellphone store
155-B West Kingsbridge Road
the Fordham Manor neighborhood
the Bronx
Feb. 7, 2019 DATE
NY Port Authority Police Department Task Force Officer ORG
K-9 Balu PRODUCT
Andres Reyes-Martinez PERSON
155-B West Kingsbridge Road TIME
the Bronx GPE
The Office of the Special Narcotics Prosecutor
a search warrant
the location
agents
officers
the cellphone store
approximately 5:50 p.m.
Reyes-Martinez
a second individual
Jesus Garcia
the only people
the store
the time
Garcia
the purported owner
the cellphone store
the counter
conversation
Reyes-Martinez
The Office of the Special Narcotics Prosecutor ORG
approximately 5:50 p.m. TIME
Reyes-Martinez PERSON
Jesus Garcia PERSON
Garcia PERSON
Reyes-Martinez PERSON
the first floor retail area
agents
officers
a second floor storage area
approximately 14,000

In [85]:
mesh_fp = '/teams/DSC180A_FA20_A00/a07opioidoverdoseprevalence/data/MESH.csv'
rxnorm_fp = '/teams/DSC180A_FA20_A00/a07opioidoverdoseprevalence/data/RXNORM.csv'
rx=pd.read_csv(rxnorm_fp,usecols=['Preferred Label','Semantic type UMLS property'])
mesh=pd.read_csv(mesh_fp,usecols=['Preferred Label','Semantic type UMLS property'])
term = pd.concat([rx,mesh])
term['Preferred Label'] = term['Preferred Label'].str.lower()
ontology = dict(zip(term['Preferred Label'],term['Semantic type UMLS property']))
terms = list(ontology.keys())
terms = terms[:50000]

In [86]:
def make_content_text(doc,doc_id):
    #Initialize a dictionary
    content = dict()       

    #Add noun to dictionary with its location as key
    for sent_id in doc.sentences:
        sent=doc.sentences[sent_id]
        for n_id,noun in enumerate(sent.noun):
            key=str(doc_id)+'sentence'+str(sent_id)+'_'+str(n_id)
            noun = noun.lower()
            if len(noun) > 3:
                content[key] = noun
               
    return content

In [87]:
def make_content_search(search_terms):
    #Initialize a dictionary
    content = dict()
    
    #Add search terms to dictionary with numbered key
    for i in range(len(search_terms)):
        if len(search_terms[i]) > 3:
            content[i] = search_terms[i].strip().lower()    
    return content

In [88]:
content = make_content_search(terms)
for doc_id,document in enumerate(doc):
    content.update(make_content_text(document,doc_id))

In [89]:
def create_lsh(content, n_permutations, n_gram):
    labels = content.keys()
    values = content.values()
    #Create MinHash object
    minhash = MinHash(values, n_gram=n_gram, permutations=n_permutations, hash_bits=64, seed=3)
    #Create LSH model
    lsh = LSH(minhash, labels, no_of_bands=5)
    return lsh

In [90]:
n_permutations= 50,
n_gram= 3
%time lsh = create_lsh(content,n_permutations,n_gram)

CPU times: user 1min 8s, sys: 283 ms, total: 1min 8s
Wall time: 1min 8s


In [91]:
edge_list = lsh.edge_list(min_jaccard=0.2,jaccard_weighted=True)

In [92]:
start = time.time()
similar_ls = []
for i in edge_list:
    if type(i[0])==str and type(i[1])==int:
        similar_ls.append(i)
        print(edge_list.index(i))
    elif (type(i[0])==int and type(i[1])==str):
        similar_ls.append(i)
        print(edge_list.index(i))
    elif type(i[0])==int and type(i[1])==int:
        break
end = time.time()
end-start

156
315
390
419
423
533
535
537
539
630
634


0.0018382072448730469

In [93]:
similar_ls

[('1sentence53_8', 23425, 1.0),
 ('1sentence29_20', 23425, 1.0),
 ('1sentence13_11', 23425, 1.0),
 ('1sentence10_1', 23425, 1.0),
 ('1sentence9_1', 23425, 1.0),
 ('0sentence35_24', 12724, 1.0),
 ('0sentence35_17', 12724, 1.0),
 ('0sentence35_5', 23425, 1.0),
 ('0sentence35_3', 8018, 1.0),
 ('0sentence26_7', 1444, 1.0),
 ('0sentence24_6', 1444, 1.0)]

In [94]:
def update_entity_type(docs,similar_ls,terms,ontology):
    for i in similar_ls:
        word_id = i[0]
        term_id = i[1]
        score = i[2]
        #get the entity 
        word = content[word_id]
        #get the sentence and word index of the entity
        doc_index=int(re.sub('(?<=sentence).+','',word_id).replace('sentence',''))
        index = re.sub('.+(?=sentence)','',word_id).replace('sentence','')
        word_index = int(re.sub('.+(?=_)','',index).replace('_',''))
        sent_index= int(re.sub('(?<=_).+','',index).replace('_',''))

        types = []
        scores = []
        # get ontology type for entity
        onto = ontology[terms[term_id]]
        types.append(onto)
        scores.append(score)
        
        #check if the noun is already in ner list
        in_ner =False
        ner_score =1
        doc = docs[doc_index]
        sent = doc.sentences[sent_index]
        #loop through list of entity dictionary for the sentence
        for entity in sent.ner:
            current = list(entity.keys())[0]
            if word==current.lower():
                types.append(entity[current])
                scores.append(ner_score)
                entity[current] = [types,scores]
                in_ner = True
                print(doc_index,sent_index)
                
        if not in_ner:
            #store type of entity to its location in the list
            sent.ner.append({word:[types,scores]})
            print(doc_index,sent_index)

In [95]:
update_entity_type(doc,similar_ls,terms,ontology)

1 53
1 29
1 13
1 10
1 9
0 35
0 35
0 35
0 35
0 26
0 24
